In [1]:
#%pip install equinox openmdao[all]

In [2]:
import equinox as eqx
from util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
opt.define_material(
    E=69e9,
    rho=2700,
    yield_strength=241e6,
    shear_strength=207e6
)

In [8]:
altitude = 2900

results = opt.solve_wing(
    lift_goal=10710,
    safety_factor=0.75,
    v_infty=57,
    rho=density(altitude),
    mu=dyn_viscousity(altitude),
    alpha_geo=jnp.deg2rad(jnp.float32(0))
)

Optimizing planform
Optimization terminated successfully    (Exit mode 0)
            Current function value: 309.87289737836437
            Iterations: 8
            Function evaluations: 14
            Gradient evaluations: 8
Optimization Complete
-----------------------------------
Optimizing airfoil
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.003735618185834414
            Iterations: 22
            Function evaluations: 30
            Gradient evaluations: 22
Optimization Complete
-----------------------------------
Optimizing spars
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.04121002375639479
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


In [9]:
from util import pretty_print

pretty_print(results)

Airfoil
----------
Base Shape Coef. 1.683
Max Thickness    0.186
Taper Exponent   2.823
Max Camber       0.000
Camber Exponent  0.948
Reflex           -0.008

Planform
----------
Wingspan         17.773 m
Chord length     1.187 m
Aspect Ratio     14.969

Spars
----------
Flange Height    0.006 m
Flange Width     0.119 m
Web Height       0.140 m
Web Width        0.006 m
Spar Ratio       7.42
Main Spar Pos.   0.05c
Rear Spar Pos.   0.95c

Aerodynamics & Structure
----------
Lift             10809
Drag             202
Cl at alpha=0    0.40
Alpha Cl=0       -3.69 deg
Max τ threshold  0.15%
Max σ threshold  9.91%
Material Usage   0.041 m3

Timing
----------
Planform         23 seconds
Airfoil          4 seconds
Spars            3 seconds
Total            30 seconds
